In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("sentiment.tsv", delimiter="\t", quoting=3)

data.head()

len(data)

25000

In [2]:
data.review[0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [8]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from contractions import fix
import re

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    
    try:
        text = fix(text)
    except Exception as e:
        print(f"Contraction error: {e} | Original text: {text}")
        return ""
    
    text = text.lower()
    
    text = re.sub(r'\d+', '', text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)

    tokens = word_tokenize(text)

    cleaned_tokens = [
        lemmatizer.lemmatize(token) for token in tokens if token not in stop_words
    ]

    return " ".join(cleaned_tokens)

In [9]:
tain_x_tum = []
for i in range(len(data["review"])):
    if(i + 1) % 1000 == 0:
        print("Process step: ", i + 1)
    tain_x_tum.append(clean_text(data["review"][i]))

Process step:  1000
Process step:  2000
Process step:  3000
Process step:  4000
Process step:  5000
Process step:  6000
Process step:  7000
Process step:  8000
Process step:  9000
Process step:  10000
Process step:  11000
Process step:  12000
Process step:  13000
Process step:  14000
Process step:  15000
Process step:  16000
Process step:  17000
Process step:  18000
Process step:  19000
Process step:  20000
Process step:  21000
Process step:  22000
Process step:  23000
Process step:  24000
Process step:  25000


In [10]:
from sklearn.model_selection import train_test_split
x = tain_x_tum
y = np.array(data["sentiment"])

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.15, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=5000)
train_x1 = vectorizer.fit_transform(train_x)
train_x1 # <21250x5000 sparse matrix of type '<class 'numpy.int64'>'

<21250x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1717825 stored elements in Compressed Sparse Row format>

In [12]:
train_x1 = train_x1.toarray()
train_y1 = train_y
train_x1.shape,train_y1.shape

((21250, 5000), (21250,))

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_x1,train_y1)

test_x1 = vectorizer.transform(test_x)
test_x1 # <3750x5000 sparse matrix of type '<class 'numpy.int64'>'

<3750x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 304677 stored elements in Compressed Sparse Row format>

In [14]:
test_x1 = test_x1.toarray()
test_x1.shape

(3750, 5000)

In [15]:
from sklearn.metrics import roc_auc_score
test_predict = rf.predict(test_x1)
dogruluk = roc_auc_score(test_y,test_predict)

In [16]:
test_predict

array([0, 1, 0, ..., 0, 0, 1], dtype=int64)

In [17]:
dogruluk

0.8455211886993982

In [ ]:
import joblib

joblib.dump(rf, "trained_sentiment.pkl")
joblib.dump(vectorizer, "sentiment_vectorizer.pkl")

In [18]:
import joblib

rf = joblib.load("trained_sentiment.pkl")
vectorizer = joblib.load("sentiment_vectorizer.pkl")

yeni_veri = ["I love this movie!"]
yeni_x = vectorizer.transform(yeni_veri).toarray()
predict = rf.predict(yeni_x)
print(predict)

[1]
